DraftKings NFL Constraint Satisfaction
===

See also [this blog post](https://levelup.gitconnected.com/dfs-lineup-optimizer-with-python-296e822a5309).

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [37]:
import os
import sys
from datetime import datetime

In [6]:
import pulp

### Load in the weekly data

In [11]:
df = pd.read_csv('DKSalaries.csv')
len(df)

470

In [12]:
df.sample(n=5)

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
211,WR,Adam Humphries (15589125),Adam Humphries,15589125,WR/FLEX,4000,Postponed,TEN,11.53
366,WR,Keelan Doss (15560732),Keelan Doss,15560732,WR/FLEX,3000,LV@KC 10/11/2020 01:00PM ET,LV,0.00
193,RB,Brian Hill (15560484),Brian Hill,15560484,RB/FLEX,4000,CAR@ATL 10/11/2020 01:00PM ET,ATL,6.95
434,TE,Reggie Gilliam (15561000),Reggie Gilliam,15561000,TE/FLEX,2500,Postponed,BUF,1.78
29,RB,Melvin Gordon III (15560298),Melvin Gordon III,15560298,RB/FLEX,6400,Postponed,DEN,17.40


In [13]:
# trim any postponed games, since those can't be included in a lineup
df = df[df['Game Info'] != 'Postponed']
len(df)

391

### Create the constraint problem

Goal: maximize AvgPointsPerGame

 - TotalPlayers = 10
 - TotalSalary <= 50000
 - TotalPosition_WR = 3
 - TotalPosition_RB = 3
 - TotalPosition_TE = 1
 - TotalPosition_QB = 1
 - TotalPosition_FLEX = 1
 - TotalPosition_DST = 1
 - Each player in only one position (relevant only for FLEX)
 

In [80]:
prob = pulp.LpProblem('DK_NFL_weekly', pulp.LpMaximize)

In [81]:
player_vars = [pulp.LpVariable(f'player_{row.ID}', cat='Binary') for row in df.itertuples()]

In [82]:
# total assigned players constraint
prob += pulp.lpSum(player_var for player_var in player_vars) == 10

In [83]:
# position constraints
# TODO fix this, currently won't work
# as it makes the problem infeasible
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['Roster Position'].iloc[i]) for i in range(len(df))])
    
prob += get_position_sum(player_vars, df, 'QB') == 1
prob += get_position_sum(player_vars, df, 'WR') == 3
prob += get_position_sum(player_vars, df, 'RB') == 3
#prob += get_position_sum(player_vars, df, 'TE') == 1
#prob += get_position_sum(player_vars, df, 'FLEX') == 1
prob += get_position_sum(player_vars, df, 'DST') == 1

In [84]:
# total salary constraint
prob += pulp.lpSum(df.Salary.iloc[i] * player_vars[i] for i in range(len(df))) <= 50000

In [85]:
# finally, specify the goal
prob += pulp.lpSum([df.AvgPointsPerGame.iloc[i] * player_vars[i] for i in range(len(df))])

In [86]:
prob.solve()

1

In [87]:
pulp.LpStatus[prob.status]

'Optimal'

In [88]:
total_salary_used = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['Roster Position'], row.Name, row.TeamAbbrev, row.Salary, row.AvgPointsPerGame)
        total_salary_used += row.Salary
total_salary_used

QB Dak Prescott DAL 7400 34.3
TE/FLEX George Kittle SF 6600 26.2
RB/FLEX Joe Mixon CIN 6300 18.32
RB/FLEX Raheem Mostert SF 6100 21.9
WR/FLEX Jamison Crowder NYJ 5800 23.95
RB/FLEX Jerick McKinnon SF 5800 16.62
DST Colts  IND 3300 13.0
WR/FLEX Jeff Smith NYJ 3000 15.1
WR/FLEX Travis Fulgham PHI 3000 13.7
TE/FLEX C.J. Uzomah CIN 2500 11.35


49800